In [1]:
import os
import sys
from pathlib import Path

import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd 
import skimage.io

from collections import defaultdict
from tqdm.auto import tqdm
from joblib import Parallel, delayed
import re
import h5py
#import napari
import tifffile as tiff
import seaborn as sns
import pickle

import networkx as nx
from sklearn.model_selection import train_test_split

C:\Users\eozturk7\AppData\Local\miniconda3\envs\torch_gpu2\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import torch_geometric.utils
import torch

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
#p_dir = (Path().cwd().parents[0]).absolute()
p_dir = 'Y:\\coskun-lab\\Efe and Nishkala\\SnowflakePipeline'

module_path = str(p_dir + "\\src")

if module_path not in sys.path:
    sys.path.append(module_path)

In [5]:
#data_dir = (Path().cwd().parents[0] / 'data').absolute()
data_dir = (p_dir + '\\Data')

In [6]:
import graph
import torch
import torch_geometric.utils
import networkx as nx
import lightning.pytorch as pl
import torch.utils.data as data

process_path = p_dir + '\\Data' + '\\torch_graph_data'

# Create data loader

In [90]:
from torch_geometric.loader import DataLoader
seed = torch.Generator().manual_seed(42)

name = 'TLS'

# Crate dataset
dataset = graph.GraphDataset(process_path + '\\' + name, process_path + '\\' + name + '\\' + 'info.csv', 2, y_name='y')

train_set, val_set, test_set = graph.train_test_val_split(dataset)

# Create Dataloader
train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
val_loader = DataLoader(val_set, batch_size=32, shuffle=True)
test_loader = DataLoader(test_set, batch_size=32, shuffle=False)


In [91]:
print(f'Dataset: {dataset}:')
print('======================')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of features: {dataset.num_features}')
print(f'Number of classes: {dataset.nc}')

Dataset: GraphDataset(34):
Number of graphs: 34
Number of features: 58
Number of classes: 2


In [92]:
print(f'Train set: {len(train_set)}, val set: {len(test_set)}, test set: {len(val_set)}')

Train set: 17, val set: 13, test set: 4


In [93]:
for step, data in enumerate(test_loader):
    print(f'Step {step + 1}:')
    print('=======')
    print(f'Number of graphs in the current batch: {data.num_graphs}')
    print(data)
    print()
    data.y
    break

Step 1:
Number of graphs in the current batch: 13
DataBatch(x=[9951, 58], edge_index=[2, 44722], pos=[9951, 2], y=[13], edge_attr=[44722, 2], name=[13], batch=[9951], ptr=[14])



# Train network K Fold

In [94]:
from lightning.pytorch.accelerators import find_usable_cuda_devices
import wandb

In [95]:
from sklearn.model_selection import KFold, StratifiedKFold, ShuffleSplit, StratifiedShuffleSplit
from torch.utils.data import SubsetRandomSampler

In [96]:
condition = 'SF_15082024_tls_snowflake_pos'
checkpoint_folder = (Path().cwd().parents[0]).absolute() / 'data' / "saved_models" / f"Graph_GNNs_{condition}" 
project_name = f'{condition}'

In [97]:
AVAIL_GPUS = [0]

# Setting the seed
pl.seed_everything(42)

NUM_LAYERS = 3
HIDDEN_CHANNELS = 32
pools = ['mean', 'max', 'attention', 'attention2']
# pools = ['attention2']
models = ['GAT', 'GINConv']
#models = ['GINConv']
epochs = 100

Seed set to 42


In [98]:
ys = [data.y.item() for data in dataset]

In [99]:
# # Print K fold model number of samples and number of positive cases

k_folds = 5
# kfold = KFold(n_splits=k_folds, shuffle=True, random_state=42)
kfold = StratifiedShuffleSplit(n_splits=k_folds, test_size=0.3, random_state=0)
val_size = 0.2

for fold, (train_ids, test_ids) in enumerate(kfold.split(dataset, ys)):
 # train_subset = dataset.index_select(train_ids.tolist())
 test_subset = dataset.index_select(test_ids.tolist())

 train_ids, valid_ids = train_test_split(train_ids, test_size = val_size, random_state=0)
 train_subset = dataset.index_select(train_ids.tolist())
 val_subset = dataset.index_select(valid_ids.tolist())

 for pool in pools:
     for model in models:
         if model == 'GAT':
             batch_size=64
         else:
             batch_size=128
         # Path to the folder where the pretrained models are saved
         CHECKPOINT_PATH = checkpoint_folder / f'{model}_{NUM_LAYERS}_{HIDDEN_CHANNELS}_{fold}' / pool
         CHECKPOINT_PATH.mkdir(parents=True, exist_ok=True)

         # Skip already trained kfold and pool
         checkpoint = CHECKPOINT_PATH / f"GraphLevel{model}" / f"GraphLevel{model}.ckpt" 
         if checkpoint.exists():
             print(checkpoint)
             continue

         # Run training
         run = wandb.init(project=project_name, name=f'{model}_{NUM_LAYERS}_{HIDDEN_CHANNELS}_{fold}', 
                         group=f'{model}_{pool}')
         graph.train_graph_classifier_kfold(model, 
                                              train_subset, 
                                              val_subset, 
                                              test_subset,
                                              dataset, 
                                              CHECKPOINT_PATH, 
                                              AVAIL_GPUS, 
                                              hidden_channels=HIDDEN_CHANNELS, 
                                              num_layers=NUM_LAYERS, 
                                              epochs=epochs,
                                              batch_size=batch_size,
                                              graph_pooling=pool,
                                              morph=False,
                                              pos=True)
         run.finish()

Seed set to 42
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Seed set to 42
Missing logger folder: Y:\coskun-lab\Efe and Nishkala\SnowflakePipeline\11_snowflakes\data\saved_models\Graph_GNNs_SF_15082024_tls_snowflake_pos_4\GAT_3_32_0\mean\GraphLevelGAT\lightning_logs
C:\Users\eozturk7\AppData\Local\miniconda3\envs\torch_gpu2\Lib\site-packages\lightning\pytorch\loggers\wandb.py:396: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
C:\Users\eozturk7\AppData\Local\miniconda3\envs\torch_gpu2\Lib\site-packages\lightning\pytorch\callbacks\model_checkpoint.py:652: Checkpoint directory \\bme-data2.ad.gatech.edu\labs5\coskun-lab\Efe and Nishkala\SnowflakePipeline\11_snowflakes\data\saved_models\Graph_GNNs_SF_15082024_tls_snowflake_pos_4\GAT_3_32_0\mean\GraphLevelGAT exists and is not empt

{'test_acc': tensor(0.6364), 'test_auc': tensor(0.6667), 'test_f1': tensor(0.7143)}


epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇████
lr-Adam,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_acc,▁
test_auc,▁
test_f1,▁
train_acc,▁▁▅▅▆▆▇▇▇███████████████████████████████
train_auc,▁▆▆▇▇███████████████████████████████████
train_f1,▁▁▆▇▇▇██████████████████████████████████
train_loss_epoch,██▇▅▄▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_loss_step,█▃▁▁▁▁▁▁▁▁
trainer/global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███


Seed set to 42
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Seed set to 42
Missing logger folder: Y:\coskun-lab\Efe and Nishkala\SnowflakePipeline\11_snowflakes\data\saved_models\Graph_GNNs_SF_15082024_tls_snowflake_pos_4\GINConv_3_32_0\mean\GraphLevelGINConv\lightning_logs
C:\Users\eozturk7\AppData\Local\miniconda3\envs\torch_gpu2\Lib\site-packages\lightning\pytorch\loggers\wandb.py:396: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
C:\Users\eozturk7\AppData\Local\miniconda3\envs\torch_gpu2\Lib\site-packages\lightning\pytorch\callbacks\model_checkpoint.py:652: Checkpoint directory \\bme-data2.ad.gatech.edu\labs5\coskun-lab\Efe and Nishkala\SnowflakePipeline\11_snowflakes\data\saved_models\Graph_GNNs_SF_15082024_tls_snowflake_pos_4\GINConv_3_32_0\mean\GraphLevelGINConv exists

{'test_acc': tensor(0.5455), 'test_auc': tensor(0.7333), 'test_f1': tensor(0.6667)}


...

{'test_acc': tensor(0.5455), 'test_auc': tensor(0.7000), 'test_f1': tensor(0.6154)}


epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇████
lr-Adam,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_acc,▁
test_auc,▁
test_f1,▁
train_acc,▁▃▄▅▅▇█▇████████████████████████████████
train_auc,▁▅▆▇▇███████████████████████████████████
train_f1,▅▁▄▆▆▇██████████████████████████████████
train_loss_epoch,██▇▆▅▄▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_loss_step,█▁▁▁▁▁▁▁▁▁
trainer/global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███


Seed set to 42
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Seed set to 42
Missing logger folder: Y:\coskun-lab\Efe and Nishkala\SnowflakePipeline\11_snowflakes\data\saved_models\Graph_GNNs_SF_15082024_tls_snowflake_pos_4\GINConv_3_32_4\attention2\GraphLevelGINConv\lightning_logs
C:\Users\eozturk7\AppData\Local\miniconda3\envs\torch_gpu2\Lib\site-packages\lightning\pytorch\loggers\wandb.py:396: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
C:\Users\eozturk7\AppData\Local\miniconda3\envs\torch_gpu2\Lib\site-packages\lightning\pytorch\callbacks\model_checkpoint.py:652: Checkpoint directory \\bme-data2.ad.gatech.edu\labs5\coskun-lab\Efe and Nishkala\SnowflakePipeline\11_snowflakes\data\saved_models\Graph_GNNs_SF_15082024_tls_snowflake_pos_4\GINConv_3_32_4\attention2\GraphLevelGI

{'test_acc': tensor(0.7273), 'test_auc': tensor(0.8667), 'test_f1': tensor(0.7273)}


epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇████
lr-Adam,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_acc,▁
test_auc,▁
test_f1,▁
train_acc,▁▃▆▆████████████████████████████████████
train_auc,▁▅▇▇████████████████████████████████████
train_f1,▁▅▇▇████████████████████████████████████
train_loss_epoch,█▆▄▄▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_loss_step,█▅▁▁▁▁▁▁▁▁
trainer/global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███


## MLP

In [100]:
AVAIL_GPUS = [0]

# Setting the seed
pl.seed_everything(42)

NUM_LAYERS = 3
HIDDEN_CHANNELS = 32
pools = ['mean', 'max', 'attention', 'attention2']
models = ['MLP']
epochs = 100

Seed set to 42


In [101]:
ys = [data.y.item() for data in dataset]

In [102]:
# Print K fold model number of samples and number of positive cases

k_folds = 5
# kfold = KFold(n_splits=k_folds, shuffle=True, random_state=42)
kfold = StratifiedShuffleSplit(n_splits=k_folds, test_size=0.3, random_state=0)
val_size = 0.2

for fold, (train_ids, test_ids) in enumerate(kfold.split(dataset, ys)):
    # train_subset = dataset.index_select(train_ids.tolist())
    test_subset = dataset.index_select(test_ids.tolist())
    
    train_ids, valid_ids = train_test_split(train_ids, test_size = val_size, random_state=0)
    train_subset = dataset.index_select(train_ids.tolist())
    val_subset = dataset.index_select(valid_ids.tolist())
    
    for pool in pools:
        for model in models:
            if model == 'GAT':
                batch_size=64
            else:
                batch_size=128
            # Path to the folder where the pretrained models are saved
            CHECKPOINT_PATH = checkpoint_folder / f'{model}_{NUM_LAYERS}_{HIDDEN_CHANNELS}_{fold}' / pool
            CHECKPOINT_PATH.mkdir(parents=True, exist_ok=True)

            # Skip already trained kfold and pool
            checkpoint = CHECKPOINT_PATH / f"GraphLevel{model}" / f"GraphLevel{model}.ckpt" 
            if checkpoint.exists():
                print(checkpoint)
                continue

            # Run training
            run = wandb.init(project=project_name, name=f'{model}_{NUM_LAYERS}_{HIDDEN_CHANNELS}_{fold}', 
                            group=f'{model}_{pool}')
            graph.train_graph_classifier_kfold(model, 
                                                 train_subset, 
                                                 val_subset, 
                                                 test_subset,
                                                 dataset, 
                                                 CHECKPOINT_PATH, 
                                                 AVAIL_GPUS, 
                                                 hidden_channels=HIDDEN_CHANNELS, 
                                                 num_layers=NUM_LAYERS, 
                                                 epochs=epochs,
                                                 batch_size=batch_size,
                                                 graph_pooling=pool,
                                                 morph=False,
                                                 pos=True)
            run.finish()

Seed set to 42
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Seed set to 42
Missing logger folder: Y:\coskun-lab\Efe and Nishkala\SnowflakePipeline\11_snowflakes\data\saved_models\Graph_GNNs_SF_15082024_tls_snowflake_pos_4\MLP_3_32_0\mean\GraphLevelMLP\lightning_logs
C:\Users\eozturk7\AppData\Local\miniconda3\envs\torch_gpu2\Lib\site-packages\lightning\pytorch\loggers\wandb.py:396: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
C:\Users\eozturk7\AppData\Local\miniconda3\envs\torch_gpu2\Lib\site-packages\lightning\pytorch\callbacks\model_checkpoint.py:652: Checkpoint directory \\bme-data2.ad.gatech.edu\labs5\coskun-lab\Efe and Nishkala\SnowflakePipeline\11_snowflakes\data\saved_models\Graph_GNNs_SF_15082024_tls_snowflake_pos_4\MLP_3_32_0\mean\GraphLevelMLP exists and is not empt

{'test_acc': tensor(0.8182), 'test_auc': tensor(0.8333), 'test_f1': tensor(0.8571)}


epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇████
lr-Adam,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_acc,▁
test_auc,▁
test_f1,▁
train_acc,▁▄▄▅▅▇▇▇████████████████████████████████
train_auc,▁▅▆▅▅▇▇▇████████████████████████████████
train_f1,▄▁▁▄▅▆▆▆████████████████████████████████
train_loss_epoch,█▇▆▆▅▅▄▄▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_loss_step,█▄▁▁▁▁▁▁▁▁
trainer/global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███


Seed set to 42
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Seed set to 42
Missing logger folder: Y:\coskun-lab\Efe and Nishkala\SnowflakePipeline\11_snowflakes\data\saved_models\Graph_GNNs_SF_15082024_tls_snowflake_pos_4\MLP_3_32_0\max\GraphLevelMLP\lightning_logs
C:\Users\eozturk7\AppData\Local\miniconda3\envs\torch_gpu2\Lib\site-packages\lightning\pytorch\loggers\wandb.py:396: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
C:\Users\eozturk7\AppData\Local\miniconda3\envs\torch_gpu2\Lib\site-packages\lightning\pytorch\callbacks\model_checkpoint.py:652: Checkpoint directory \\bme-data2.ad.gatech.edu\labs5\coskun-lab\Efe and Nishkala\SnowflakePipeline\11_snowflakes\data\saved_models\Graph_GNNs_SF_15082024_tls_snowflake_pos_4\MLP_3_32_0\max\GraphLevelMLP exists and is not empty.

{'test_acc': tensor(0.4545), 'test_auc': tensor(0.4667), 'test_f1': tensor(0.)}


...

{'test_acc': tensor(0.4545), 'test_auc': tensor(0.6000), 'test_f1': tensor(0.)}


epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇████
lr-Adam,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_acc,▁
test_auc,▁
test_f1,▁
train_acc,▁▃▃▂▂▂▄▅▆▄▄▄▅▄▃▂▄▇▄▄▂▇▃▇▄▅▄▇▇▆█▆▅▅▅▄▅▇▇▄
train_auc,▃▅▃▁▁▃▃▅█▆▆▄▄▇▆▃▇▇▆▆▄█▆▇▆▇▆█▇▆██▆▇█▇▆▇▇▆
train_f1,▅▅▃▁▁▁▅▆▆▄▃▅▆▆▅▄▄▇▆▅▂▇▄▇▅▆▅▇▇▆█▆▆▆▆▅▆▇▇▆
train_loss_epoch,█▅▅▆▇▆▅▄▃▄▅▅▅▅▅▆▃▃▄▅▆▂▄▂▃▃▄▂▃▃▁▃▃▄▃▄▄▂▂▄
train_loss_step,▆█▅▇▅▁▂▁▂▅
trainer/global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███


Seed set to 42
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Seed set to 42
Missing logger folder: Y:\coskun-lab\Efe and Nishkala\SnowflakePipeline\11_snowflakes\data\saved_models\Graph_GNNs_SF_15082024_tls_snowflake_pos_4\MLP_3_32_4\attention2\GraphLevelMLP\lightning_logs
C:\Users\eozturk7\AppData\Local\miniconda3\envs\torch_gpu2\Lib\site-packages\lightning\pytorch\loggers\wandb.py:396: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
C:\Users\eozturk7\AppData\Local\miniconda3\envs\torch_gpu2\Lib\site-packages\lightning\pytorch\callbacks\model_checkpoint.py:652: Checkpoint directory \\bme-data2.ad.gatech.edu\labs5\coskun-lab\Efe and Nishkala\SnowflakePipeline\11_snowflakes\data\saved_models\Graph_GNNs_SF_15082024_tls_snowflake_pos_4\MLP_3_32_4\attention2\GraphLevelMLP exists and

{'test_acc': tensor(0.6364), 'test_auc': tensor(0.7333), 'test_f1': tensor(0.7143)}


epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇████
lr-Adam,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_acc,▁
test_auc,▁
test_f1,▁
train_acc,▁▁▃▅▃▅▆▆▅▆▆▇█▆▇██▇████████████████████▇█
train_auc,▁▂▆▇▇▆██▇███████████████████████████████
train_f1,▁▁▄▆▅▆▇▇▆▇▆██▇██████████████████████████
train_loss_epoch,██▇▆▅▅▃▃▄▄▄▂▁▃▂▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_loss_step,█▅▅▂▁▁▂▁▁▂
trainer/global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
